### Hussmann_et_al_2022

This notebook analyses tracking files derived from the "Fiji Manual" plugin, and produces plots that are displayed in the manuscript.

### Imports and Functions

In [ ]:
# Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# Graphic Settings
sns.set_palette("Set1", 8, .75)
sns.set_theme(style="whitegrid")
plt.tight_layout()

# Used functions

def calculate_distance(df):
    """Calculates distance between consecutive tracking points and returns the distance in microns."""
    
    diff_x = df.X.diff()
    diff_y = df.Y.diff()
    dist = np.sqrt((diff_x*diff_x) + (diff_y*diff_y))
    dist = dist*df.Pixelsize_micron
    
    return dist


def calculate_velocity(df):
    """Calculates cell velocity."""
    
    vel = df.Distance/df.Timeinterval_min
    
    return vel


def calculate_centered(df):
    """Creates a track with a centered (0,0) start point."""
    
    df["X_centered"] = -1
    df["Y_centered"] = -1
    for i in df.Track_ID.unique():
    #print(i)
    #print(df.Track == i)
        df.X_centered[df.Track_ID == i] = df[df.Track_ID == i].iloc[:, 2] - df[df.Track_ID == i].iloc[0, 2]
        df.Y_centered[df.Track_ID == i] = (df[df.Track_ID == i].iloc[:, 3] - df[df.Track_ID == i].iloc[0, 3])*-1
        
    return df


def make_summary_df(df):
    """Creates a summary of all measurements."""
    
    df_summary = pd.DataFrame()
    df_summary["Track_ID"] = df.groupby(["Track_ID"]).mean().index
    df_summary = df_summary.set_index("Track_ID")
    df_summary["Mean_velocity_per_track"] = df.groupby(["Track_ID"]).mean().Velocity
    df_summary["Median_velocity_per_track"] = df.groupby("Track_ID").median().Velocity
    df_summary["Total_distance_per_track"] = df.groupby("Track_ID").sum().Distance
    df_summary["Y_migration_distance_per_track"] = df.groupby(["Track_ID"]).Y_centered.apply(lambda y: y.tail(1).values[-1]*df.groupby(["Track_ID"]).Pixelsize_micron.mean().values[-1])
    df_summary["Phenotype"] = df.groupby("Track_ID").Phenotype.apply(lambda pheno: pheno.unique()[0])
    
    return df_summary


def plot_general(df_summary, save_directory="\\", save=False):
    """Plots measurements as violinplots."""
    
    for i in range(len(df_summary.columns)-1):
        #print(i)
        fig = plt.figure(figsize=(8,5))
        ax = fig.add_subplot(111)
        ax = sns.violinplot(y=df_summary.iloc[:,i], x=df_summary.Phenotype, data=df_summary, hue=df_summary.Phenotype, width=0.7, dodge=False)
        handles, labels = ax.get_legend_handles_labels()
        
        lgd = ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
        ax.set_title(df_summary.columns[i])
        if save: 
            fig.savefig(save_directory + "\\" + df_summary.columns[i] + ".svg", dpi=300, format="svg",  bbox_inches='tight')
            
        fig.show()
    if save:
        print("Your data hase been saved to:", save_directory)
            

### Open individual .csv files

In [ ]:
#Type in your directories here:
directory = #Insert directory here
save_directory = #Insert saving directory here

In [ ]:
#TO DO: Add example data to GitHub, incl. Metadata.xlsx

files = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        files.append(os.path.join(directory, filename))

print("You will open the tracking results from", len(files)-1, " movies.")
    
metadata = pd.read_excel(directory + "\Metadata.xlsx")
print("Your metadatafile contains:", metadata.shape[0], "movies.")

if len(files) != metadata.shape[0]:
    print("Your metadatafile does not contain all movies!")

### Start Analysis

In [ ]:
df = pd.DataFrame(columns=['Track n', 'Slice n', 'X', 'Y', 'Distance', 'Velocity', 'Pixel Value',
       'Filename', 'Movie_ID', 'Pixelsize_micron', 'Timeinterval_min',
       'Phenotype'])

for i in files:
    if i.endswith("Metadata.csv") == False:
        #print(i)
        df1 = pd.read_csv(i, delimiter=",",encoding_errors="ignore")
        df1["Filename"] = i.split('\\')[-1]
        df1 = df1.merge(metadata, how="inner", left_on="Filename", right_on="Filename")
        meta = metadata[metadata["Filename"] == i.split('\\')[-1]]
        movie_id = meta.Movie_ID.values[0]
        pixelsize = meta.Pixelsize_micron.values[0]
        timeinterval = meta.Timeinterval_min.values[0]
        df1["Track_ID"] = df1['Movie_ID'].astype(str) + "_"+ df1['Track n'].astype(str)
        df1["Distance"] = calculate_distance(df1)
        df1["Velocity"] = calculate_velocity(df1)
        df = df.append(df1, ignore_index=True)
df = calculate_centered(df)

## Save the whole table
df.to_excel(save_directory +"\Results_all.xlsx")

### Summary table

In [ ]:
df_summary = make_summary_df(df)
df_summary.head()

In [ ]:
# Save the summary
df_summary.to_excel(save_directory +"\Summary_table.xlsx")

### Graphical representation 

In [ ]:
plot_general(df_summary, save_directory=save_directory, save=True)

#### Centered coordinates

In [ ]:
sns.relplot(x=df.X_centered, y=df.Y_centered, hue=df.Track_ID, data=df, kind="line", sort=False, ci=None, palette="viridis", col=df.Phenotype)
plt.savefig(save_directory + "\\Centered_graphs.svg", dpi=300)

In [ ]:
sns.lineplot(x=df.X_centered, y=df.Y_centered, hue=df.Phenotype, data=df, sort=False, ci=None, units=df.Track_ID, estimator=None)

#### Plotting migration distance over time

In [ ]:
#Calculation cumulative distance over time
df["Cumulative_distance"] = 0

for i in df.Track_ID.unique():
    for x in df[df.Track_ID == i].index:
        if x == df[df.Track_ID == i].index.min():
            #print(x, "is smallest")
            df.iloc[x, -1] = 0
        else:
            df.iloc[x, -1] = (df.iloc[x, 4] + df.iloc[x-1, -1])

In [ ]:
sns.relplot(x=df["Slice n"]*df.Timeinterval_min, y=df.Cumulative_distance, data=df, hue=df.Track_ID, kind="line", col=df.Phenotype) #
plt.xlabel("Time (min)")
plt.ylabel("Cumulative_Distance (um)")